In [9]:
import requests
import xml.etree.ElementTree as ET

# 1. 발급받은 API 키를 여기에 입력하세요
service_key = "50hjvXuloV4qFNdrIUOglZZ6RGV7uq7pvpP0oxT+EV57bvEGnWfvqbjL939z/yfj9ta/H2Cn382mGmHpm4wmcw=="

# 2. 새로운 API 요청 URL
url = "http://apis.data.go.kr/1471000/CsmtcsIngdCpntInfoService01/getCsmtcsIngdCpntInfoService01"

# 3. 요청에 필요한 파라미터 설정
# 'INGR_KOR_NAME'을 추가하여 특정 원료를 검색할 수 있습니다.
params = {
    "serviceKey": service_key,
    "pageNo": 1,
    "numOfRows": 10,
    # "INGR_KOR_NAME": "비타민" # 특정 원료를 검색하려면 이 주석을 해제하세요
}

try:
    # 4. GET 요청 보내기
    print("API 요청을 보냅니다...")
    response = requests.get(url, params=params)
    response.raise_for_status()

    # 5. XML 응답 파싱
    root = ET.fromstring(response.content)
    
    # 6. XML에서 데이터 추출하여 출력
    # XML 태그 이름이 대문자로 바뀌었으므로, 이에 맞게 수정합니다.
    items = root.findall('.//item')

    if items:
        print("--- 검색 결과 (XML) ---")
        for item in items:
            ingr_name_tag = item.find('INGR_KOR_NAME')
            ingr_eng_name_tag = item.find('INGR_ENG_NAME')
            cas_no_tag = item.find('CAS_NO')
            
            ingr_name = ingr_name_tag.text if ingr_name_tag is not None else "정보 없음"
            ingr_eng_name = ingr_eng_name_tag.text if ingr_eng_name_tag is not None else "정보 없음"
            cas_no = cas_no_tag.text if cas_no_tag is not None else "정보 없음"
            
            print(f"원료명: {ingr_name}")
            print(f"영문명: {ingr_eng_name}")
            print(f"CAS No: {cas_no}")
            print("-" * 20)
    else:
        print("검색 결과가 없습니다.")

except requests.exceptions.RequestException as e:
    print(f"요청 중 오류 발생: {e}")
except ET.ParseError as e:
    print(f"XML 파싱 오류: {e}")

API 요청을 보냅니다...
--- 검색 결과 (XML) ---
원료명: 가공소금
영문명: None
CAS No: None
--------------------
원료명: 가지열매추출물
영문명: Solanum Melongena (Eggplant) Fruit Extract
CAS No: 84012-19-1
--------------------
원료명: 구멍쇠미역추출물
영문명: Agarum Cribrosum Extract
CAS No: None
--------------------
원료명: 루핀아미노산
영문명: Lupine Amino Acids
CAS No: None
--------------------
원료명: 류신
영문명: Leucine
,61-90-5(L-)39-2(DL-)
--------------------
원료명: 류코노스톡/무발효여과물
영문명: Leuconostoc/Radish Root Ferment Filtrate
CAS No: None
--------------------
원료명: 백혈구추출물
영문명: Leukocyte Extract
CAS No: None
--------------------
원료명: 리나칸투스 콤무니스추출물
영문명: Rhinacanthus Communis Extract
CAS No: None
--------------------
원료명: 리날룰
영문명: Linalool
CAS No: 78-70-6
--------------------
원료명: 리날릴아세테이트
영문명: Linalyl Acetate
CAS No: 115-95-7
--------------------
